Network Features

In [1]:
import sys
sys.path.append('../scripts')
from utils import *

ROOT PATH: ../


In [2]:
class CascadeAnalyzer(object):
    feature_df = pd.DataFrame()  # output

    def __init__(self):
        self.meta_df = pd.DataFrame()  # labels / key: root_tweet_id
        self.cascades_dict = {}  # key: root_tweet_id, value: Cascade()
        self.retrieve_cascade_labels()
        self.load_cascades()

    def retrieve_cascade_labels(self):
        column_names = ['label', 'tweet_id']
        self.meta_df = pd.read_csv(DATA_PATH + "label.txt", sep=':', names=column_names, converters={'tweet_id': str})
        print("-------------------------------------")
        print(self.meta_df.info())
        print("-------------------------------------" * 2)
        print(self.meta_df.shape, self.meta_df['label'].value_counts().to_dict())
        print("-------------------------------------" * 2)
        print(self.meta_df.head())
        print("-------------------------------------\n")

    def load_cascades(self):
        # TODO: handle pickle data
        # iterate tweet trees
        for index, file in enumerate(os.listdir(DATA_PATH + 'tree_u')):
            if not file.endswith('.txt'):
                print("Unexpected Input File:", file)
                continue
            root_tweet_id = file.replace('.txt', '')  # file_id
            cascade_path = os.path.join(DATA_PATH + 'tree_u', file)
            label = self.meta_df.loc[self.meta_df['tweet_id'] == root_tweet_id, 'label'].item()  # label
            self.cascades_dict[root_tweet_id] = Cascade(root_tweet_id, cascade_path, label)
            print(self.cascades_dict[root_tweet_id])

    # Main Outer loop
    def iterate_cascades(self):
        for index, row in self.meta_df.iterrows():
            tweet_id = row['tweet_id']
            cascade = self.cascades_dict[tweet_id]
            print('#', index, row['tweet_id'], row['label'])
            cascade.calc_structural_features()

    def cascade_to_csv(self):  # CascadeAnalyzer
        ensure_directory(OUT_PATH)
        out_file_name = OUT_PATH + 'structural_analysis_' + time.strftime("%Y%m%d_%H%M%S") + ".csv"
        out_file = open(out_file_name, 'w', encoding='utf-8', newline='')
        self.feature_df.to_csv(out_file, sep=',', index=False)

In [3]:
class Cascade:

    # --------------------------
    #      Initiate Cascade
    # --------------------------
    def __init__(self, root_tweet_id, cascade_path, label=None):
        self.file_id = root_tweet_id  # For label.txt
        self.root_tweet_id = root_tweet_id  # Tweet ID with ROOT Keyword (May updated)
        self.root_user_id = 0
        self.cascade_path = cascade_path
        self.label = label

        # ------------
        # Load Cascade
        # ------------
        self.trace_count = None
        self.src_users = set()
        self.dst_users = set()
        self.retweet_users = set()
        self.reply_users = set()
        self.retweet_count = 0
        self.reply_count = 0
        self.network = nx.DiGraph()
        self.network_features = {}
        self.load_cascade()

        # -----------------
        # Calculate Cascade
        # -----------------
        self.src_user_count = None
        self.dst_user_count = None
        self.avg_depth = 0
        self.max_depth = 0

    def load_cascade(self):
        with open(self.cascade_path, 'r') as file:
            # ---- -----------------
            # Set Root: User, Tweet
            # ---------------------
            for index, line in enumerate(file):
                elem_list = [x.strip() for x in re.split(r"[\'\,\->\[\]]", line.strip()) if x.strip()]
                if elem_list[0] == 'ROOT' and elem_list[1] == 'ROOT':
                    self.root_user_id = elem_list[3]
                    if index != 0:
                        print('ROOT TWEET {} by {} @ line # {}'.format(elem_list[4], self.root_user_id, index))
                    break
            if self.root_tweet_id != elem_list[4]:  # Assert file_id == root_tweet_id
                print('\t file_id:{1} -> root_tweet_id:{2} ({0}) '.format(self.label, self.root_tweet_id, elem_list[4]))
                self.root_tweet_id = elem_list[4]
            # ------------
            # Load Cascade
            # ------------
            for index, line in enumerate(file):  # Trace
                elem_list = re.split(r"[\'\,\->\[\]]", line.strip())
                elem_list = [x.strip() for x in elem_list if x.strip()]  # Remove empty elements
                # Error data handling
                if float(elem_list[2]) >= float(elem_list[5]):
                    continue
                src_user_id, src_tweet_id, src_tweet_time, dst_user_id, dst_tweet_id, dst_tweet_time = elem_list
                self.src_users.add(src_user_id)
                self.dst_users.add(dst_user_id)
                # Different types of Tweets - https://help.twitter.com/en/using-twitter/types-of-tweets
                if src_tweet_id == dst_tweet_id:
                    self.retweet_count += 1
                    self.retweet_users.add(dst_user_id)
                else:
                    self.reply_count += 1
                    self.reply_users.add(dst_user_id)
                # NetworkX Graph
                self.network.add_weighted_edges_from(
                    [(src_user_id, dst_user_id, float(dst_tweet_time) - float(src_tweet_time))])
        # Store computed cascade information
        self.trace_count = index

    # =============================
    #      Structural Analysis
    # =============================
    def calc_structural_features(self):
        G = self.network
        # root_user_id = self.root_user_id
        self.src_user_count = len(self.src_users)
        self.dst_user_count = len(self.dst_users)
        hops = []
        max_hop_count = 10
        for i in range(max_hop_count):
            hops.append(len(nx.single_source_shortest_path_length(G, self.root_user_id, cutoff=i)))

        # print(self.retweet_count, self.response_count)
        # print("leaf:", nx.dag_to_branching(G))
        # print('\t root_to_all_depth_length: ', len(nx.single_source_shortest_path_length(G, self.root_user_id)))
        # print('\t user_count:', len(G.nodes()))  # root + dst_user_count
        print('\t depth: ', nx.dag_longest_path(G))  # weight - temporal feature
        print('\t src_user_count: ', self.src_user_count)
        print('\t dst_user_count: ', self.dst_user_count)
        print('\t root_to_all_depth_sum: ', sum(nx.single_source_shortest_path_length(G, self.root_user_id).values()))
        print('\t root_to_all_depth_max: ', max(nx.single_source_shortest_path_length(G, self.root_user_id).values()))
        print('\t one_hop_neighbors:', len(list(G.neighbors(self.root_user_id))))
        print('\t', "user count by hop(s): ", hops[1] - hops[0], hops[2] - hops[1], hops[3] - hops[2],
                                              hops[4] - hops[3], hops[5] - hops[4], hops[6] - hops[5],
                                              hops[7] - hops[6], hops[8] - hops[7], hops[9] - hops[8])

        # df.loc[df['tweet_id'] == root_tweet_id, 'src_user_count'] = len(src_users)
        shortest_path_dict = nx.single_source_shortest_path_length(G, self.root_user_id)
        self.avg_depth = sum(shortest_path_dict.values()) / len(shortest_path_dict)
        self.max_depth = max(shortest_path_dict.values())

        for i in range(max_hop_count - 1):
            self.network_features[str(i) + "_hop_neighbor_count"] = hops[i + 1] - hops[i]



        # features to data frame
        CascadeAnalyzer.feature_df = CascadeAnalyzer.feature_df.append({
            'tweet_id': self.root_tweet_id, 'label': self.label,
            'structural_trace_count': self.trace_count,
            'structural_retweet_count': self.retweet_count,
            'structural_reply_count': self.reply_count,
            'structural_src_user_count': self.src_user_count,
            'structural_dst_user_count': self.dst_user_count,
            'structural_retweet_reply_percent': self.retweet_count / (self.retweet_count + self.reply_count),
            'structural_src_dst_user_percent': self.src_user_count / (self.src_user_count + self.dst_user_count),  # <--
            'structural_retweet_users_count': len(self.retweet_users),
            'structural_reply_users_count': len(self.reply_users),
            'structural_root_to_all_depth_sum': sum(nx.single_source_shortest_path_length(G, self.root_user_id).values()),
            'structural_root_to_all_depth_max': max(nx.single_source_shortest_path_length(G, self.root_user_id).values()),
            'structural_1_hop_neighbor_count': self.network_features['1_hop_neighbor_count'],
            'structural_2_hop_neighbor_count': self.network_features['2_hop_neighbor_count'],
            'structural_3_hop_neighbor_count': self.network_features['3_hop_neighbor_count'],
            'structural_4_hop_neighbor_count': self.network_features['4_hop_neighbor_count'],
            'structural_5_hop_neighbor_count': self.network_features['5_hop_neighbor_count'],
            'structural_6_hop_neighbor_count': self.network_features['6_hop_neighbor_count'],
            'structural_7_hop_neighbor_count': self.network_features['7_hop_neighbor_count'],
            'structural_8_hop_neighbor_count': self.network_features['8_hop_neighbor_count'],
            'structural_avg_depth': self.avg_depth,
            'structural_max_depth': self.max_depth,  # duplicate
            # 'structural_max_depth': self.max_depth,  # duplicate
            'structural_network_density': nx.density(G),  # duplicate
        }, ignore_index=True)


In [4]:
analyzer = CascadeAnalyzer()
analyzer.iterate_cascades()
analyzer.cascade_to_csv()

-------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 818 entries, 0 to 817
Data columns (total 2 columns):
label       818 non-null object
tweet_id    818 non-null object
dtypes: object(2)
memory usage: 12.9+ KB
None
--------------------------------------------------------------------------
(818, 2) {'non-rumor': 205, 'false': 205, 'true': 207, 'unverified': 201}
--------------------------------------------------------------------------
        label            tweet_id
0       false  656955120626880512
1        true  615689290706595840
2       false  613404935003217920
3  unverified  731166399389962242
4  unverified  714598641827246081
-------------------------------------

ROOT TWEET 715253497462145024 by 8839632 @ line # 17
	 file_id:715255507506892800 -> root_tweet_id:715253497462145024 (unverified) 
ROOT TWEET 692692529779707904 by 5402612 @ line # 20
	 file_id:692702295281262593 -> root_tweet_id:692692529779707904 (non-rumor) 
ROOT TWEET 7529342

ROOT TWEET 692742353736568833 by 5402612 @ line # 94
	 file_id:692753210692476928 -> root_tweet_id:692742353736568833 (non-rumor) 
ROOT TWEET 690676539323740161 by 5402612 @ line # 17
	 file_id:690680149164085248 -> root_tweet_id:690676539323740161 (non-rumor) 
ROOT TWEET 742053606820438016 by 56300259 @ line # 8
	 file_id:745365403237376000 -> root_tweet_id:742053606820438016 (unverified) 


ROOT TWEET 693069637030957057 by 5402612 @ line # 22
	 file_id:693087220459270144 -> root_tweet_id:693069637030957057 (non-rumor) 


ROOT TWEET 692664097125601281 by 3646911 @ line # 2
	 file_id:692665281362202624 -> root_tweet_id:692664097125601281 (non-rumor) 
ROOT TWEET 766808183696351233 by 373564351 @ line # 1


ROOT TWEET 692742353736568833 by 5402612 @ line # 26
	 file_id:692748411481780224 -> root_tweet_id:692742353736568833 (non-rumor) 
# 0 656955120626880512 false
	 depth:  ['1942819082', '2985968438']
	 src_user_count:  52
	 dst_user_count:  521
	 root_to_all_depth_sum:  598
	 root_to_all_depth_max:  3
	 one_hop_neighbors: 454
	 user count by hop(s):  454 57 10 0 0 0 0 0 0
# 1 615689290706595840 true
	 depth:  ['44945327', '494338530']
	 src_user_count:  13
	 dst_user_count:  356
	 root_to_all_depth_sum:  366
	 root_to_all_depth_max:  2
	 one_hop_neighbors: 346
	 user count by hop(s):  346 10 0 0 0 0 0 0 0
# 2 613404935003217920 false
	 depth:  ['14511951', '1173625512']
	 src_user_count:  5
	 dst_user_count:  117
	 root_to_all_depth_sum:  120
	 root_to_all_depth_max:  2
	 one_hop_neighbors: 114
	 user count by hop(s):  114 3 0 0 0 0 0 0 0
# 3 731166399389962242 unverified
	 depth:  ['431917957', '705461595749683200']
	 src_user_count:  13
	 dst_user_count:  138
	 root_to_all_depth_sum: 

...

# 654 614594259900080128 true
	 depth:  ['1917731', '630818269']
	 src_user_count:  21
	 dst_user_count:  168
	 root_to_all_depth_sum:  198
	 root_to_all_depth_max:  3
	 one_hop_neighbors: 140
	 user count by hop(s):  140 26 2 0 0 0 0 0 0
# 655 614614133410033664 true
	 depth:  ['14173315', '1308116874', '1307998092']
	 src_user_count:  73
	 dst_user_count:  643
	 root_to_all_depth_sum:  777
	 root_to_all_depth_max:  4
	 one_hop_neighbors: 523
	 user count by hop(s):  523 107 12 1 0 0 0 0 0
# 656 544278335455776769 true
	 depth:  ['380648579', '29018320', '551342421']
	 src_user_count:  28
	 dst_user_count:  212
	 root_to_all_depth_sum:  266
	 root_to_all_depth_max:  4
	 one_hop_neighbors: 165
	 user count by hop(s):  165 41 5 1 0 0 0 0 0
# 657 665309822208729088 non-rumor
	 depth:  ['15108702', '835747213']
	 src_user_count:  36
	 dst_user_count:  259
	 root_to_all_depth_sum:  321
	 root_to_all_depth_max:  3
	 one_hop_neighbors: 205
	 user count by hop(s):  205 46 8 0 0 0 0 0 0
# 658 

	 depth:  ['5402612', '187475837', '3022334712']
	 src_user_count:  72
	 dst_user_count:  1340
	 root_to_all_depth_sum:  1538
	 root_to_all_depth_max:  4
	 one_hop_neighbors: 1155
	 user count by hop(s):  1155 175 7 3 0 0 0 0 0
# 689 723504069814444033 unverified
	 depth:  ['111556701', '392593149']
	 src_user_count:  19
	 dst_user_count:  213
	 root_to_all_depth_sum:  296
	 root_to_all_depth_max:  4
	 one_hop_neighbors: 144
	 user count by hop(s):  144 57 10 2 0 0 0 0 0
# 690 553586860334010368 true
	 depth:  ['1299769218', '435777178', '2471375798']
	 src_user_count:  44
	 dst_user_count:  268
	 root_to_all_depth_sum:  374
	 root_to_all_depth_max:  4
	 one_hop_neighbors: 178
	 user count by hop(s):  178 76 12 2 0 0 0 0 0
# 691 640118021101604864 unverified
	 depth:  ['20562637', '159320818']
	 src_user_count:  8
	 dst_user_count:  121
	 root_to_all_depth_sum:  139
	 root_to_all_depth_max:  4
	 one_hop_neighbors: 108
	 user count by hop(s):  108 9 3 1 0 0 0 0 0
# 692 68875106150344294

# 724 614616994499788800 true
	 depth:  ['288215711', '866804630']
	 src_user_count:  27
	 dst_user_count:  251
	 root_to_all_depth_sum:  286
	 root_to_all_depth_max:  4
	 one_hop_neighbors: 223
	 user count by hop(s):  223 22 5 1 0 0 0 0 0
# 725 693185853867294721 non-rumor
	 depth:  ['759251', '54400569', '735097672089149441']
	 src_user_count:  199
	 dst_user_count:  1145
	 root_to_all_depth_sum:  1709
	 root_to_all_depth_max:  7
	 one_hop_neighbors: 837
	 user count by hop(s):  837 190 53 22 19 18 6 0 0
# 726 553506608203169792 true
	 depth:  ['5402612', '535765349']
	 src_user_count:  32
	 dst_user_count:  561
	 root_to_all_depth_sum:  623
	 root_to_all_depth_max:  4
	 one_hop_neighbors: 513
	 user count by hop(s):  513 37 8 3 0 0 0 0 0
# 727 751856580874960897 unverified
	 depth:  ['198286584', '573842268', '20479813', '3691110204']
	 src_user_count:  11
	 dst_user_count:  165
	 root_to_all_depth_sum:  334
	 root_to_all_depth_max:  4
	 one_hop_neighbors: 56
	 user count by hop(s)

	 root_to_all_depth_max:  3
	 one_hop_neighbors: 2197
	 user count by hop(s):  2197 215 6 0 0 0 0 0 0
# 760 686666933949837312 non-rumor
	 depth:  ['788524', '79030269']
	 src_user_count:  97
	 dst_user_count:  667
	 root_to_all_depth_sum:  1241
	 root_to_all_depth_max:  7
	 one_hop_neighbors: 325
	 user count by hop(s):  325 197 90 36 8 2 8 0 0
# 761 767710042816602112 non-rumor
	 depth:  ['1367531', '4675424281']
	 src_user_count:  40
	 dst_user_count:  953
	 root_to_all_depth_sum:  1005
	 root_to_all_depth_max:  3
	 one_hop_neighbors: 906
	 user count by hop(s):  906 42 5 0 0 0 0 0 0
# 762 714811995573325828 unverified
	 depth:  ['20748745', '2306191482']
	 src_user_count:  9
	 dst_user_count:  125
	 root_to_all_depth_sum:  133
	 root_to_all_depth_max:  2
	 one_hop_neighbors: 117
	 user count by hop(s):  117 8 0 0 0 0 0 0 0
# 763 728101712762834944 unverified
	 depth:  ['2267865715', '724544443517263872']
	 src_user_count:  10
	 dst_user_count:  143
	 root_to_all_depth_sum:  151
	 r

	 depth:  ['16664681', '265705962', '1163873018']
	 src_user_count:  181
	 dst_user_count:  1045
	 root_to_all_depth_sum:  1787
	 root_to_all_depth_max:  6
	 one_hop_neighbors: 581
	 user count by hop(s):  581 309 54 82 16 3 0 0 0
# 799 552806757672964097 true
	 depth:  ['15754281', '14728168', '2479977919']
	 src_user_count:  29
	 dst_user_count:  258
	 root_to_all_depth_sum:  333
	 root_to_all_depth_max:  4
	 one_hop_neighbors: 196
	 user count by hop(s):  196 50 11 1 0 0 0 0 0
# 800 693761289601060864 non-rumor
	 depth:  ['5402612', '132155424']
	 src_user_count:  49
	 dst_user_count:  934
	 root_to_all_depth_sum:  1264
	 root_to_all_depth_max:  4
	 one_hop_neighbors: 624
	 user count by hop(s):  624 294 12 4 0 0 0 0 0
# 801 544350567183556608 true
	 depth:  ['25210650', '774393817', '318588826']
	 src_user_count:  14
	 dst_user_count:  118
	 root_to_all_depth_sum:  177
	 root_to_all_depth_max:  4
	 one_hop_neighbors: 64
	 user count by hop(s):  64 51 1 2 0 0 0 0 0
# 802 75882553548

In [5]:
analyzer.cascades_dict

{'498430783699554305': <__main__.Cascade at 0x7f57af77ad68>,
 '500378223977721856': <__main__.Cascade at 0x7f57adcc3080>,
 '524922729485848576': <__main__.Cascade at 0x7f57acb37860>,
 '524923462398513152': <__main__.Cascade at 0x7f57ad856ac8>,
 '524923676484177920': <__main__.Cascade at 0x7f57aa1d1d30>,
 '524925050739490816': <__main__.Cascade at 0x7f57a804f198>,
 '524925215235911680': <__main__.Cascade at 0x7f57ac839668>,
 '524925987239120897': <__main__.Cascade at 0x7f57aac5cd68>,
 '524926235030589440': <__main__.Cascade at 0x7f57a870e358>,
 '524931324763992064': <__main__.Cascade at 0x7f57ad36bd30>,
 '524932935137628160': <__main__.Cascade at 0x7f57aa3c9470>,
 '524935485370929152': <__main__.Cascade at 0x7f57a9deaa58>,
 '524936872666353664': <__main__.Cascade at 0x7f57ab1e0c88>,
 '524941132237910016': <__main__.Cascade at 0x7f57a7fa3f98>,
 '524942470472548352': <__main__.Cascade at 0x7f57ac0bc9e8>,
 '524943490887991296': <__main__.Cascade at 0x7f57ad46a7f0>,
 '524944399890124801': <

<module 'networkx' from '/home/jihochoi/.local/lib/python3.5/site-packages/networkx/__init__.py'>

In [15]:
G = analyzer.cascades_dict['498430783699554305'].network
G2 = analyzer.cascades_dict['673615400655970304'].network

In [16]:
np.mean(list(nx.betweenness_centrality(G).values()))  # average betweenness centrality

8.023025552008068e-06

In [17]:
np.mean(list(nx.betweenness_centrality(G2).values()))

2.771960397654025e-07

In [21]:
analyzer.cascades_dict['498430783699554305'].root_user_id

'24165761'

In [23]:
sum(nx.single_source_shortest_path_length(G, analyzer.cascades_dict['498430783699554305'].root_user_id).values()) / analyzer.cascades_dict['498430783699554305'].dst_user_count

1.4360902255639099

In [25]:
sum(nx.single_source_shortest_path_length(G2, analyzer.cascades_dict['673615400655970304'].root_user_id).values()) / analyzer.cascades_dict['673615400655970304'].dst_user_count

1.1038338658146964